## What is ETL?

ETL is actually short form of Extract, Transform and Load, a process in which data is acquired, changed/processes and then finally get loaded into data warehouse/database(s).

You can extract data from data sources like Files, Website or some Database, transform the acquired data and then load the final version into database for business usage.

You may ask, Why ETL?, well, what ETL does, many of you might already been doing one way or other by writing different functions/scripts to perform tasks but one of the main advantage of ETLs is visualizing your entire data flow pipeline thus help you make decisions according to that.

Let's start with building our own ETL pipeline.
* Extract data from CSV file
* Transform/Manipulate Data
* Load Data into MongoDB

In [1]:
# to read data from csv, python provides csv module
import csv

To deal with files in Python, we use the open() function, it’s a built-in Python function. This function accepts two different arguments (inputs) in the parentheses, always in the following order:
* the name of the file (as a string)
* the mode of working with the file (as a string)

The syntax to open a file in python is:

file_obj = open(“filename”, “mode”)  

In [3]:
f = open(r'/content/crypto-markets.csv')
# 'f' is a file handler here

csv_reader = csv.reader(f)
print(csv_reader)


![title](Crypto_Market_Data.PNG)

Transforming/Changing the data.

In [4]:
assetsCode = ['BTC','ETH','XRP','LTC']

# initialize empty list
crypto_data = []

next(csv_reader, None)  # skips the headers

# read csv data row wise
for row in csv_reader:
    if(row[1] in assetsCode):  # check if current row consist of either 'BTC' or 'ETH' or 'XRP' or 'LTC' currency data
        # convert open, high, low, close amount to float type first from str and then convert it into GBP
        row[5] = float(row[5]) * 0.75
        row[6] = float(row[6]) * 0.75
        row[7] = float(row[7])* 0.75
        row[8] = float(row[8]) * 0.75
        crypto_data.append(row)

# print(csv_reader.line_num)
print(len(crypto_data))
print(crypto_data[0:2])

7239
[['bitcoin', 'BTC', 'Bitcoin', '2013-04-28', '1', 101.47500000000001, 101.98499999999999, 99.07499999999999, 100.6575, '0', '1488566728', '0.5438', '3.88'], ['bitcoin', 'BTC', 'Bitcoin', '2013-04-29', '1', 100.83, 110.6175, 100.5, 108.405, '0', '1603768865', '0.7813', '13.49']]


Loading the data into SQL DB

In [6]:
import sqlite3

# connect function opens a connection to the SQLite database file,
conn = sqlite3.connect('session.db')
#Similarly we will make connection with other databases like Oracle, DB2 etc.

In [7]:
# Drop a table name Crypto id it exists already
try:
    conn.execute('DROP TABLE IF EXISTS `Crypto` ')
except Exception as e:
    print(str(e))

In [8]:
# Create a new Table named as Crypto
try:
    conn.execute('''
         CREATE TABLE Crypto
         (ID         INTEGER PRIMARY KEY,
         NAME        TEXT    NOT NULL,
         Date        datetime,
         Open        Float DEFAULT 0,
         High        Float DEFAULT 0,
         Low         Float DEFAULT 0,
         Close       Float DEFAULT 0);''')
    print ("Table created successfully");
except Exception as e:
    print(str(e))
    print('Table Creation Failed!!!!!')
finally:
    conn.close() # this closes the database connection

Table created successfully


In [9]:
# Since our crypto data contains more information than required so we need eliminate some of it.
print(crypto_data[0])

['bitcoin', 'BTC', 'Bitcoin', '2013-04-28', '1', 101.47500000000001, 101.98499999999999, 99.07499999999999, 100.6575, '0', '1488566728', '0.5438', '3.88']


In [10]:
# Some more transformations
crypto_sql_data = [(row[2], row[3], row[5], row[6], row[7], row[8]) for row in crypto_data]
crypto_sql_data[:2]

[('Bitcoin',
  '2013-04-28',
  101.47500000000001,
  101.98499999999999,
  99.07499999999999,
  100.6575),
 ('Bitcoin', '2013-04-29', 100.83, 110.6175, 100.5, 108.405)]

In [11]:
# lets make new connection to Insert crypto data in SQL DB
conn = sqlite3.connect('session.db')
cur = conn.cursor()
try:
    cur.executemany("INSERT INTO Crypto(NAME, Date, Open, High, Low, Close) VALUES (?,?,?,?,?,?)", crypto_sql_data)
    conn.commit()
    print('Data Inserted Successfully')
except Exception as e:
    print(str(e))
    print('Data Insertion Failed')
finally:
    conn.close()

Data Inserted Successfully


In [12]:
# Let's Read data from DB to verify it

conn = sqlite3.connect('session.db')
rows = conn.cursor().execute('Select * from Crypto')

for row in rows:
    print(row)

Выходные данные были обрезаны до нескольких последних строк (5000).
(2240, 'XRP', '2014-02-17', 0.0114675, 0.01196925, 0.011319, 0.0114705)
(2241, 'XRP', '2014-02-18', 0.011481, 0.0117735, 0.0112395, 0.011464499999999999)
(2242, 'XRP', '2014-02-19', 0.01143225, 0.01170525, 0.011291250000000001, 0.01142775)
(2243, 'XRP', '2014-02-20', 0.011398499999999999, 0.01161375, 0.010629, 0.01068825)
(2244, 'XRP', '2014-02-21', 0.0107265, 0.01140225, 0.010298999999999999, 0.0113025)
(2245, 'XRP', '2014-02-22', 0.01132725, 0.01180575, 0.01093725, 0.011761500000000001)
(2246, 'XRP', '2014-02-23', 0.011707500000000001, 0.012057, 0.011511, 0.011559)
(2247, 'XRP', '2014-02-24', 0.011558249999999999, 0.01165725, 0.0101925, 0.0101925)
(2248, 'XRP', '2014-02-25', 0.010203750000000001, 0.0102525, 0.00812925, 0.0097425)
(2249, 'XRP', '2014-02-26', 0.009764249999999999, 0.01063575, 0.009764249999999999, 0.01056975)
(2250, 'XRP', '2014-02-27', 0.01059, 0.01075575, 0.010269, 0.010614750000000001)
(2251, 'XRP',

Write data in a csv file

In [13]:
csvfile = open('Crypto.csv', 'w')
csv_writer = csv.writer(csvfile, lineterminator='\r')
# Now we can write data to files using two methods:
# writerow() or writerows()
# writerow() is used when we need to write one-dimension data such as a single list :[1, ‘Jerry’, 95]
# writerows() is used when we need to write multi-dimension data such as list of list [[1, ‘Jerry’, 95], [2, ‘Tom’, 80], [3, ‘Scooby’, 90]]
# So the only difference is that writerows() lets you pass multiple values!
csv_writer.writerow(['Name', 'Date', 'Open', 'High', 'Low', 'Close'])
csv_writer.writerows(crypto_sql_data)
csvfile.close()